Trabalho II de Tópicos Especiais III

In [ ]:
!pip install tweet-preprocessor
!pip install sklearn

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from collections import Counter
from sklearn.preprocessing import LabelEncoder

from nltk.cluster import KMeansClusterer
import nltk
from sklearn import cluster
from sklearn import metrics

import preprocessor as pre
from sklearn.preprocessing import scale

from google.colab import files
import io

import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering


In [ ]:
# Modulo para fazer a carga dos dados de Tweets
# Deve fazer o upload do arquivo primeiro

url = '/content/Tweets.csv'

tweets = pd.read_csv(url)

#visualizar tweets
tweets

In [ ]:
# Print o exemplo de comentarios do tweet (sujo)
comments = tweets['text']
comments

In [ ]:
# Limpeza usando biblioteca tweet-preprocessor:URL, EMOJI, HASHTAG, RESERVADO, SMILE, NUMEROS E AUTHOR 
pre.set_options(pre.OPT.URL, pre.OPT.EMOJI, pre.OPT.HASHTAG, pre.OPT.RESERVED, pre.OPT.SMILEY, pre.OPT.NUMBER, pre.OPT.MENTION)

comments_process = []
for linha in comments:
  linha = pre.clean(linha)
  linha = linha.replace('$',' ')
  linha = linha.replace('?',' ')
  linha = linha.replace('-',' ')
  linha = linha.replace('.',' ')
  linha = linha.replace('!',' ')
  linha = linha.replace('/','')
  comments_process.append(pre.clean(linha))
comments_process

In [ ]:
#Importar modelo do USE diretamente do tensorflow hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)

#função que recebe os tweets pré-processados e retorna os embeddings para todos os tweets
def embed(input):
    return model(input)

In [ ]:
# Gera o embeddings dos tweets
tweets_embeddings = embed(comments_process)
tweets_embeddings.shape

In [ ]:
# Apenas para testar 
for i, tweet_embedding in enumerate(np.array(tweets_embeddings).tolist()):
    print("Message: {}".format(comments_process[i]))
#    print("Embedding size: {}".format(len(tweet_embedding)))
    message_embedding_snippet = ", ".join((str(x) for x in tweet_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

In [ ]:
# Plotagem do dendograma
from scipy.cluster import hierarchy
dendrogram = sch.dendrogram(sch.linkage(tweets_embeddings, method="average", metric='cosine'), p=20, truncate_mode='level')
plt.title('Dendrograma')
plt.xlabel('Tweets')
plt.ylabel('Proximidade por Coseno')
plt.show()

In [ ]:
# Clusterizando
cluster = AgglomerativeClustering(n_clusters=200, affinity='cosine', linkage='average')
result = cluster.fit_predict(tweets_embeddings)
cluster

In [ ]:
# Print labels
result

In [ ]:
# Estrutura Data Frame clusterizado
resultado = pd.DataFrame({'ID':result,
                        'Comment':comments_process})
resultado

In [ ]:
# Verificando grupos
counter = Counter(resultado['ID'])
counter

In [ ]:
# Teste de proximidade
for linha in resultado.iterrows():
  if linha[1][0] ==168:
    print(linha[1][1] + '\n\n')

In [ ]:
# Teste de proximidade
for linha in resultado.iterrows():
  if linha[1][0] ==54:
    print(linha[1][1] + '\n\n')